# 1. Introduction:

This notebook demonstrates -

1. Use of DPU and Preprocessing Accelerators to perform AI Inference on video input.
2. Option to run different AI models like facedetect, refinedet, and SSD using the same set of accelerators.

The application is based on the VVAS (Vitis Video Analytics SDK) framework, also utilizing the open source GStreamer plugins.

Vitis Video Analytics SDK (VVAS) is developed by Xilinx to provide many useful GStreamer plugins as the middleware between the application and underlying FPGA accelerators, including DPU AI inference engine, and other PL accelerators such as the one for AI input preprocessing.

Please refer to the [Kria™ KV260 Vision AI Starter Kit Applications GitHub Pages](https://xilinx.github.io/kria-apps-docs/index.html) for detailed HW/SW architecture and [Vitis Video Analytics SDK GitHub Pages](https://xilinx.github.io/VVAS/#) for the VVAS related info.

# 2. Import modules

In [ ]:
import IPython
from IPython.display import *
import ipywidgets as widgets
from ipywidgets import *
import time       

# 3. Video Pipeline

Pipeline flow-

The VCU decode the H264 file to NV12 format.
The preprocessing block resizes, quantizes, and converts to BGR format.
DPU does the AI inference based on the model selected and generates bounding box data.
Meta Affixer scales the bounding box data recieved wrt the resolution of vcu decoded output.
The bounding box draws the results around the objects of interest.

PL - Pre-proc, DPU

Hardened Block - VCU Dec, Display Port

In [ ]:
from IPython.display import Image
Image(filename='./images/ml-inference.png')

* If DPU is in slot0 and PP_PIPELINE is in slot1. Copy the dpu_slot0 xclbin

In [ ]:
! cp /lib/firmware/xilinx/k26-dfx-2rp/DPU/DPU_slot0/dpu_RP_0.xclbin /lib/firmware/xilinx/k26-dfx-2rp/DPU
! mv /lib/firmware/xilinx/k26-dfx-2rp/DPU/dpu_RP_0.xclbin /lib/firmware/xilinx/k26-dfx-2rp/DPU/dfx-ml-inference.xclbin
! cp /lib/firmware/xilinx/k26-dfx-2rp/DPU/DPU_slot0/dpu_RP_0.xclbin /usr/lib
! mv /usr/lib/dpu_RP_0.xclbin /usr/lib/dpu.xclbin

* If DPU is in slot1 and PP_PIPELINE is in slot0. Copy the dpu_slot1 xclbin

In [ ]:
! cp /lib/firmware/xilinx/k26-dfx-2rp/DPU/DPU_slot1/dpu_RP_1.xclbin /lib/firmware/xilinx/k26-dfx-2rp/DPU
! mv /lib/firmware/xilinx/k26-dfx-2rp/DPU/dpu_RP_1.xclbin /lib/firmware/xilinx/k26-dfx-2rp/DPU/dfx-ml-inference.xclbin
! cp /lib/firmware/xilinx/k26-dfx-2rp/DPU/DPU_slot1/dpu_RP_1.xclbin /usr/lib
! mv /usr/lib/dpu_RP_1.xclbin /usr/lib/dpu.xclbin

# 4. Choose AI Inference Model

* Option to set required AI model

In [ ]:
aitask= [ "refinedet","facedetect","ssd" ]

In [ ]:
w = widgets.Dropdown(options=aitask,value=aitask[0],description='ai_task :',)
ai_task = aitask[0]
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global ai_task 
        ai_task = change['new']
w.observe(on_change)
display(w)

* Running Input media Sample Video

In [ ]:
if ai_task == 'ssd':
    src = './Road-Adas.nv12.h264'
else:
    src = './walking-people.nv12.h264'

In [ ]:
! gst-launch-1.0 filesrc location={src} ! h264parse ! omxh264dec !  tee name=t0 t0.src_0 !  queue !  kmssink bus-id=fd4a0000.display  sync=false fullscreen-overlay=true > /dev/null 2>&1

# 5. Run AI Inference

In [ ]:
confdir="/opt/xilinx/dfx-ml-inference/vvas/config/"+ ai_task

Bounding box will come on the region of interest on the monitor depending upon the chosen model. 

Pipeline running on Input file-

gst-launch-1.0 filesrc location={src} ! h264parse ! omxh264dec ! tee name=t0 ! queue ! vvas_xmultisrc kconfig="{confdir}/preprocess.json" ! vvas_xfilter kernels-config="{confdir}/aiinference.json" ! scalem0.sink_master vvas_xmetaaffixer name=scalem0 scalem0.src_master ! fakesink t0. ! queue max-size-buffers=1 leaky=2 ! scalem0.sink_slave_0 scalem0.src_slave_0 ! queue ! vvas_xfilter kernels-config="{confdir}/drawresult.json" ! queue ! kmssink  bus-id=fd4a0000.display sync=false fullscreen-overlay=true

In [ ]:
! gst-launch-1.0 filesrc location={src} ! h264parse ! omxh264dec ! tee name=t0 ! queue ! vvas_xmultisrc kconfig="{confdir}/preprocess.json" ! vvas_xfilter kernels-config="{confdir}/aiinference.json" ! scalem0.sink_master vvas_xmetaaffixer name=scalem0 scalem0.src_master ! fakesink t0. ! queue max-size-buffers=1 leaky=2 ! scalem0.sink_slave_0 scalem0.src_slave_0 ! queue ! vvas_xfilter kernels-config="{confdir}/drawresult.json" ! queue ! kmssink bus-id=fd4a0000.display sync=false fullscreen-overlay=true > /dev/null 2>&1

# 6. Summary
The Jupyter application shows how to use two accelerators-DPU and PP_PIPELINE in different slots to do inference on the incoming frames, and draw boundboxing of detected results.
Both accelerators work in both slots.

<center>Copyright© 2023 Xilinx</center>